# Algorithms

### Glossary 

**Student** : Someone trying to learn a course.  
**Nature / Universe**: An entity that knows the best parameters to achieve optimal reward. Its receives prediction from the learning algorithm. It calculates expectated reward for the choosen arm. Sends its to Bernoulli, which returns a reward. It sends feedback to the learning algorithm to update its parameters. 
**Learning Algorithm** : It predicts the best content for a student. It sends its prediction to nature & gets the a feedback.  It updates the parameters of the arm to incorporate the feedback. Its does this iteratively to be close to the optimal policy. 
**Arms/Content/Action** : For this notebook, these words mean the same. Its the _content_ presented to the student, or the _arm_ pulled by the bandit algorithm or the _action_ taken to obtain a reward.  
**Course**: A subject a student is trying to master.  
**Topic**: A concept within a course. Every topic has atleast 2 ways of teaching a concept.  
**Content**: A way of conveying knowledge to the student, to streamline learning. This is choosen by the learning algorithm.  
**Feedback/Reward**: A response send by the student to indicate whether they found content useful. A recognition of the quality of predictions made by the learning algorithm, which tries to maximize it.  
**Rounds(T)**: Total number of rounds played all students to complete a course. Parameters are updated in every round.   

Nature estimates expected reward for different arms. If knows the best parameters, but does not reveal it to the learning algorithm. It calculates the expected reward for each arm as $E[{r_{t,a}}|{x_{t,a}}] =  x^{T}_{t,a} θ^∗_{a}$ , where _'r'_ is the expected reward for predicting an action '_a_'  at round 't' given context information about the student & content is equal to the dot product of the transpose of the context vector and the parametric vector $\theta$ of an arm.

### Flow


### TO-DO

- MAKE SURE YOU MULTIPLYING THE RIGHT CONTEXT. A WITH A , V WITH V . ORDER NEEDS TO BE CONSISTENT. 
- NATURE: NORMALIZING THE PARAMETERS , OVER THE CONTEXT,  FOR EACH ARM
- Multithreading across students to simulate real life scenario. 
- Relevant features for content. More features for student & content. 
- Criteria to trust the skipping predictor. 
- Prototype system , with user interface showing the learning algorithm in action.

In [245]:
# $E[{r_{t,a}}|{x_{t,a}}] =  x^{T}_{t,a} θ^∗_{a}$ 
import numpy as np
import pandas as pd
from scipy.stats import bernoulli
#from sklearn.preprocessing import LabelEncoder

# Variables

In [246]:
# class AccessVariable:
#     var_to_access = 10
    
#     def my_func(self):
#         print("var_to_access : ", self.var_to_access)
    
# av = AccessVariable()
# av.my_func()

# my_dict = {'a': [1] , 'b' : [2,4], 'c': [3,5,6]}
# my_dict.values()

# no_contents_per_topic = np.random.randint(2,5,number_of_topics)
# sum(no_contents_per_topic)


# Contexts

In [247]:
# class DummyClass:
        
#     def func_1(self):
#         self.var1 = 5
        
#     def func_2(self):
#         print(self.var1)

# dc = DummyClass()
# dc.func_1()
# dc.func_2()

In [263]:
'''
This class holds student data. Its made to have all attributes of the students. StudentContext takes a subset of attributes 
of a student
'''
class Students:
#     def __init__(self):
#         self.studentsFeatures = '' # Can be avoid initliazation OR find suitable initialization. 
    '''
    student_data created during data generation
    '''
    def setStudentsFeatures(self , student_data):
#         studentDataGenerator = StudentDataGen()
#         studentsFeatures = studentDataGenerator.create()
        self.studentsFeatures = student_data
    
    def getStudentsFeatures(self):
        return self.studentsFeatures
    
'''
This class holds content data. Its made to have all attributes of contents & topics. ContentContext takes a subset of 
attributes of a content & topics
'''
    
class Content:
#     def __init__(self):
#         self.contentsFeatures = '' # Can be avoid initliazation OR find suitable initialization. 
#         self.contentDataGenerator = ContentDataGen()
        
    def getCourseContent(self): # Rename to content data
#         self.contentsFeatures = self.contentDataGenerator.getContentsFeatures() 
#         self.topicContent = self.contentDataGenerator.getTopicContent()
        return self.contentsFeatures
    '''
    courseContent created during data generation
    '''
    def setCourseContent(self,courseContent):
        self.contentsFeatures = courseContent
        
    def getTopics(self):
        return self.topicContent
    '''
    topics created during data generation
    '''   
    def setTopics(self,topics):
        self.topicContent = topics

'''
Parent class for student & content data generators. Its uses the StudentDataGen & ContentDataGen to create data. 
'''
class DataGenerator:
    
    def __init__(self):
        self.studentDataGen = StudentDataGen()
        self.contentDataGenerator = ContentDataGen()
        
    def createStudentData(self):
        self.studentData =  self.studentDataGen.create()

    def getStudentData(self):
        return self.studentData
    
    def createContentData(self):
        self.contentsFeatures = self.contentDataGenerator.getContentsFeatures() 
        self.topicContent = self.contentDataGenerator.getTopicContent()
        
    def getContentData(self):
        return self.contentsFeatures
    
    def getTopicData(self):
        return self.topicContent

'''
This is the student data generator
'''
class StudentDataGen:
    def __init__(self):
        self.number_of_students = 2 # Students taking the course. 
        self.student_context = ['video','audio','reading','kinesthetic'] # Student preferences
        # Have student preferences & probability of having those preferences as a tuple. 
    
    def create(self):
        ## Create Student Context Data
        student_context_df = pd.DataFrame(data=np.random.binomial(1 , [0.7,0.6,0.5,0.4] , 
                            size=(self.number_of_students,len(self.student_context))) , columns = self.student_context)
        return student_context_df
    
'''
This is the content data generator
'''
class ContentDataGen:
    
    def __init__(self):
        self.number_of_topics = 5 # Number of topics in the course
        self.content_context = ['A','B','C','D','E','F'] # Content features. Add meaningful features.
        self.prob_content_context = [0.8,0.7,0.6,0.5,0.4,0.3]
        #content_columns = ["content_id" , "encoded" , "prior_prob" , "rewards" , "rejections"]
#         self.content_columns = ["content_id" , "encoded" , "prior_prob" , "rewards" , "rejections"]
        # no_contents_per_topic : This can be a constant or variable. Comment the one you don't want to use
        # no_contents_per_topic = [4] * number_of_topics # Same number of contents per topic
        self.no_contents_per_topic = np.random.randint(2,5,number_of_topics) # Variable number of contents per topic.
    #     topic_content = {} # Maps topics to content. 
    #     all_topics = [] # Saves all topics for this course. 
    #     all_contents = [] # Saves all content we have for the course
    
    def create(self):
        all_contents = list()
        topic_content = {}
        for i,j in enumerate(self.no_contents_per_topic):
            topic_id = "T_" + str(i+1) # e.g : T_10
            #all_topics.append(topic_id)
            content_ids = [] # Temporary variable to help map topic to content. 
            for j_1 in range(1,j+1) : # Number of contents
                c_id = 'C_' + str(i+1) + '_' + str(j_1) # e.g : C_10_2 : Content number 2 for topics 10
                content_ids.append(c_id)
                all_contents.append(c_id)
            topic_content[topic_id] = content_ids
            #le = LabelEncoder().fit(all_contents)    
        return topic_content , all_contents
    
    # Content related features
    def getContentsFeatures(self):
        self.topic_content , self.all_contents = self.create()
        content_context_df = pd.DataFrame(data=np.random.binomial(1 , self.prob_content_context, 
                             size=(sum(self.no_contents_per_topic),len(self.content_context))) , 
                             columns = self.content_context , index=self.all_contents)
        return content_context_df
    
    def getTopicContent(self):
        return self.topic_content

'''
Context data for learning
'''
class Context:
#    def __init__(self):
#         self.students = Students()
#         self.content = Content()
        
    def getStudentContext(self):
        return self.studentContext
#         return self.students.getStudentsFeatures()
    
    def setStudentContext(self , studentFeatures):
        self.studentContext = studentFeatures
        #return self.students.getStudentsFeatures()
    
    def getContentContext(self):
        return self.contentContext
#         return self.content.getCourseContent()
    
    def setContentContext(self , courseContent):
        self.contentContext = courseContent
        #         return self.content.getCourseContent()
    
    
#     def getContent(self):
#         return self.content
    
#     def getStudents(self):
#         return self.students
    
#context = Context().getContext()
# print("self.student_context_df : " , context.student_context_df)
# print("*************************************************************")
# print("self.course_context_df : ", context.course_context_df )
#return self

# Nature / Universe

In [249]:
# p = np.random.uniform(size=(10 , 9))
# print(p)
# for i in range(p.shape[0]):
#     p[i] = p[i] / np.sum(p[i])
# print("*********************************************")
# print(p)

In [250]:
# df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
#                     index=['cobra', 'viper', 'sidewinder'],
#                     columns=['max_speed', 'shield'])
# type(df.loc['cobra'])



In [251]:
class Nature : 
    
#     def __init__(self):
# #         Initialization
#         self.theta_df = self.setParameters()
    '''
    arms: Content ids
    number_of_contexts: Number of dimensions
    '''
# CHECK: NISHANT: NORMALIZING THE PARAMETERS , OVER THE CONTEXT,  FOR EACH ARM
    def setParameters(self, contexts , arms):
        parameters = np.random.uniform(size=(len(arms) , len(contexts)))
        # Normalize parameters
        for i in range(parameters.shape[0]): # Have it in a list comprehension.
            parameters[i] = parameters[i] / np.sum(parameters[i])
        self.theta_df = pd.DataFrame(data = parameters ,  index = arms , columns = contexts , dtype= np.float)
#         print(self.theta_df)
#         print('**************************************')
#         print(self.theta_df.loc['C_1_1'])
        #return self.theta_df #This line is just for experimenting. Not part of main working code. So. REMOVE IT AFTER TESTING.
    
#     '''
#     X: Context information. 
#     arm_id: Id of the arm pulled. 
#     '''
    def getReward(self,X,arm_id):
        arm_theta = self.theta_df.loc[arm_id] #Get parameters for the arm predicted by the learning algo
        print('Arm Theta: ', arm_theta)
        expected_reward = pd.Series.dot(X,arm_theta)
        print('expected_reward : ', expected_reward)
        #expected_reward = np.dot(X.T,arm_theta) # Vector dim : (1 * d) (d * 1). Check out vector multiplication. 
        reward = bernoulli.rvs(size=1,p=expected_reward)[0] # Simulate student's response
        print('Actual Reward : ', reward)
        return reward
    
    
        


In [252]:
# import pandas as pd
# series_1 = pd.Series(data=[1,1,1,0] , index=['A','B','C','D'])
# series_2 = pd.Series(data=[1,1,1,1] , index=['D','C','B','A'])
# pd.Series.dot(series_1,series_2)

In [253]:
# p = np.random.uniform(size=(10 , 5))
# type(p[0])

In [254]:
# df = pd.DataFrame({'col_1' : [1,2,3] , 'col_2' : ['a','b','c']})
# df.index.tolist()
# list_1 = [1,2,3]
# list_2 = [4,5,6]
# list_3 = list_1 + list_2
# list_3

# contexts = ['Video','Audio','Reading','Kinesthetics','A','B','C','D','E','F']
# arms = ['C_1_1','C_1_2','C_1_3','C_2_1']
# nature = Nature()
# theta_df = nature.setParameters(contexts,arms)
#print(theta_df)
#print(theta_df.loc['C_1_1'])



In [255]:
#contexts = ['Video','Audio','Reading','Kinesthetics','A','B','C','D','E','F']
# values = [1,1,0,1,1,0,1,1,0,0]
# data_point_series = pd.Series(values , index=contexts)
# print(data_point_series.values)
# nature.getReward(data_point_series,'C_1_2')

[1 1 0 1 1 0 1 1 0 0]


In [256]:
# If I modify a list in a function, would it change the original list too. 

# class TestClass:
    
#     def my_func(self,m_l):
#         m_l.remove(2)

# my_list = [1,2,3]
# tc = TestClass()
# tc.my_func(my_list.copy())
# print(my_list)



[1, 2, 3]


In [257]:
# a = np.zeros((10, 2))
# b = a.T
# c = b.view()
# c.reshape(5,4)
# a = np.arange(6).reshape(3,2)
# np.ravel(a)

array([0, 1, 2, 3, 4, 5])

In [258]:
#Condition to check if list is empty

# a = []
# if not a:
#     print("List is empty")
# else:
#     print("List not empty")

List is empty


In [338]:
# Playing with dot product

# arr_1 = np.array([1,2,3])
# arr_2 = np.array([4,5,6])
# print('arr_1.shape : ' , arr_1.shape)
# print('arr_1.T.shape : ' , arr_1.T.shape)
# np.dot(arr_1,arr_2)

# I = np.eye(3)
# arr_1 = np.array([1,2,3])
# #arr_1 = arr_1.reshape(3,1)
# result = np.dot(arr_1,I)
# result.shape

# df = pd.DataFrame({'col_1' : [1,2,3,4] , 'col_2' : [5,6,7,8]})
# for index , context in df.iterrows():
# #     print(index)
#     print(context * 2)

col_1     2
col_2    10
Name: 0, dtype: int64
col_1     4
col_2    12
Name: 1, dtype: int64
col_1     6
col_2    14
Name: 2, dtype: int64
col_1     8
col_2    16
Name: 3, dtype: int64


In [377]:
# Learning Algorithm (LinUCB)
class LinUCB:
    def __init__(self,alpha=0.5):
        self.alpha = alpha # Hyper parameter required for LinUCB to adjust confidence bounds.
        self.arm_params = {} # Maps content to arm object
        self.rounds = 0
        self.rounds_data = pd.DataFrame()
        
#       for s in number_of_students:
#             for t in topic_content.keys():
#                 content_ids = topic_content[t]
#                 for c_id in content_ids: # Can you list comprehension to make code compact
#                     if c_id not in content_arms:
#                         a = Arm(number_of_features)
#                         content_arms[c_id] = a
#                     # Prepare context for this round. It comprises of content & student features. 
#                     X = getContext(s,c_id)
#                     # Get theta for each content id
#                     arm = content_arms[c_id]
#                     theta = getTheta(arm)
                    
    '''
    Method called by Simulator. Encapsulates finding the best arm, making a prediction, getting rewards & updating parameters. 
    '''
    #def learn(self, studentContext , contentContext ,arms , nature):
    def learn(self, context ,arms , nature):
#         X = pd.Series() #Context Vector
#         print('contentContext : ', contentContext)
        while(True):
            arms_payoff = list() # Check if the payoff values change for arms that have not been pulled. 
            for arm in arms: # arms is a list of all arms available w.r.t content
                X = context.loc[arm]
                print('X : ', X)
#                 for index , context in X.iterrows():
                print('Number of dimensions : ', len(X.index))
#                 print('studentContext : ', type(studentContext))
#                 print('contentContext.loc[arm]] : ', type(contentContext.loc[arm]))
#                 X = pd.concat([studentContext,contentContext.loc[arm]] , axis=1)
#                 print('X : ', X)
#                 print('Dimensions : ', len(X.columns))
                if arm not in self.arm_params:
                    self.arm_params[arm] = Arm(len(X.index))
                theta = self.getTheta(self.arm_params[arm])
                pta = self.getPta(theta , X , self.arm_params[arm])
                arms_payoff.append(pta)
            print('arms_payoff : ', arms_payoff)
            arm_index = np.argmax(arms_payoff) 
            print('Index of arm with max payoff : ', arm_index)
            arm_pulled = arms[arm_index]
            print('Arm pulled : ', arm_pulled)
            real_payoff = nature.getReward(X,arm_pulled)
            self.rounds += 1
            arm_obj = self.arm_params[arm_pulled]
            arm_obj.updateParams(X,real_payoff)
            self.record_rounds_data(X, np.max(arms_payoff), np.sum(arms_payoff) - np.max(arms_payoff), real_payoff)
            if real_payoff == 0:
                arms.remove(arm_pulled) # Remove that arm from the list
                if not arms: # Checking if there are arms that can be pulled. 
                    break
            else:
                break
            
        
    '''
       arm: Arm object
    '''
    def getTheta(self,arm): # This method would be within fit
        #Ainv = np.linalg.inv(arm.A)
        theta = np.dot(arm.Ainv , arm.b)
#         print('theta.shape : ', theta.shape) #Should be a column vector.
        return theta
    
    def getMean(self, context , arm):
        #theta = getTheta(arm)
#         print('context.shape : ', context.shape)
#         print('arm.theta.T.shape : ', arm.theta.T.shape)
        mean = np.dot(arm.theta.T , context)
        print('mean : {0} type : {1}'.format(mean,type(mean)))
        return mean
        
    def getUCB(self , arm , context):
        #Ainv = np.linalg.inv(arm.A)
        ucb = np.sqrt(np.dot(np.dot(context.T , arm.Ainv) , context))
        return ucb
    
    def getPta(self, theta, context , arm):
        payoff = self.getMean(context,arm) + self.alpha * self.getUCB(arm,context)
        return payoff
    
    '''
    studentContext: students contextual data
    expected_reward: expected reward of arm played
    sum_of_arm_rewards: rewards for remaining arms, that have not been played
    reward: actual reward received
    '''
    def record_rounds_data(self, studentContext, expected_reward , sum_of_arm_rewards, reward):
        r_data = {'expected_reward': expected_reward , 'potential_reward' : sum_of_arm_rewards , 'reward': reward}
        series = pd.Series()
        series = series.append([studentContext,pd.Series(r_data)])
        print('series in record_rounds_data: ' , series)
        #series = pd.concat([studentContext , pd.Series(r_data)])
        self.rounds_data = self.rounds_data.append(series , ignore_index=True)
    
    def getRoundsData(self):
        return self.rounds_data
    
#     def predict():
    
#     def update():
        
class Arm:
    def __init__(self,dimensions):
        self.A = np.identity(dimensions)
        self.b = np.zeros(dimensions)
        self.Ainv = np.linalg.inv(self.A)
        self.theta = np.dot(self.Ainv , self.b)
    
    def updateParams(self, context, reward):
        self.A += np.outer(context,context.T)
        self.b += reward * context
        self.Ainv = np.linalg.inv(self.A)
        

In [378]:
#Convert a dictionary to a series 

# my_dict = {'col_1' : 1, 'col_2' : 2}
# pd.Series(my_dict)

# df = pd.DataFrame({'col_1' : [1,2,3,4] , 'col_2' : ['a','b','c','d']})
# for index , entry in df.iterrows():
#     print(entry['col_1'])
#list(df.columns)
# df_1 = df.loc[[0,2]]
# df_1.loc[2]

# series_1 = pd.Series({'video':1 , 'audio': 1 , 'reading':1 , 'kinesthetics' : 0})
# series_2 = pd.Series({'A':1 , 'B':0})
# #pd.concat([series_1,series_2] , axis=1 , ignore_index=True , sort=True)
# X_1 = pd.Series()
# X_1 = X_1.append(series_1)
# X_1 = X_1.append(series_2)
# X_1
#series_2 = pd.Series

In [379]:
# stu = pd.Series(dict(video=[1,0,1,1] , audio = [0,0,1,1]))
# stu

In [380]:
class Simulator:
    
    def __init__(self):
        self.dataGenerator = DataGenerator()
        self.dataGenerator.createStudentData()
        self.dataGenerator.createContentData()
        self.students = Students()
        self.students.setStudentsFeatures(self.dataGenerator.getStudentData())
        self.contents = Content()
        self.contents.setCourseContent(self.dataGenerator.getContentData())
        self.contents.setTopics(self.dataGenerator.getTopicData())
        self.contexts = Context()
        self.contexts.setStudentContext(self.students.getStudentsFeatures())
        self.contexts.setContentContext(self.contents.getCourseContent())
        self.nature = Nature()
        self.linUCB = LinUCB()
        
            
    def main(self):
        studentContext = self.contexts.getStudentContext() # Student dataframe
        contentContext = self.contexts.getContentContext() # Content Dataframe
        topics = self.contents.getTopics()
        #number_of_dimensions = len(studentContext.columns) + len(contentContext.columns)
        contexts = list(studentContext.columns) + list(contentContext.columns)
#         print('Contexts : ' , contexts)
        self.nature.setParameters(contexts , contentContext.index)
        
        for index , student in studentContext.iterrows():
            for t in topics:
                content = topics[t] # You now have all arm associated with the topic 't'
                #self.learn(student , contentContext ,content)
                X = pd.DataFrame()
                for c in content:
                    print('c : ', c)
                    c_c = contentContext.loc[c]
                    X_c = pd.Series()
                    X_c = X_c.append([student,c_c])
#                     X_c = X_c.append(c_c)
                    X_c['Content_id'] = c
                    X = X.append(X_c , ignore_index=True)
                X.set_index('Content_id' , inplace=True)
#                 print(X)
#                 print('****************************')
#                 print(X.loc['C_5_1'])
                    
                #self.linUCB.learn(student , contentContext.loc[content] ,content.copy(), self.nature)
                self.linUCB.learn(X ,content.copy(), self.nature) # Remove content.copy(), as you can fetch content from X. Contend_id is now the index. 
                # Now we need to find the best arm
        print('Rounds Data: ')
        print(self.linUCB.getRoundsData())
        print('Total Number of rounds : ', self.linUCB.rounds)
        
#         print(self.students.getStudentsFeatures())
#         print("*****************************************")
#         print(self.contents.getCourseContent())
#         print("*****************************************")
#         print(self.contents.getTopics())
#         print("*****************************************")
#         print(self.contexts.getStudentContext())
#         print("*****************************************")
#         print(self.contexts.getContentContext())


#         features = self.context.getStudentContext().columns + self.context.getStudentContext().columns # Concatenate lists
#         contents = context.course_context_df.index
#         topics = self.contents.getTopics()
# # print("self.student_context_df : " , context.student_context_df)
# # print("*************************************************************")
# # print("self.course_context_df : ", context.course_context_df )
#         nature = Nature()
#         nature.setParameters(len(features),contents)
#         linUCB = LinUCB()
#         for student in context.student_context_df
#             for topic in 


simulator = Simulator()
simulator.main()

c :  C_5_1
c :  C_5_2
X :  A              0.0
B              1.0
C              1.0
D              1.0
E              0.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        0.0
video          1.0
Name: C_5_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              1.0
E              0.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        0.0
video          1.0
Name: C_5_2, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
arms_payoff :  [1.118033988749895, 1.224744871391589]
Index of arm with max payoff :  1
Arm pulled :  C_5_2
Arm Theta:  video          0.134301
audio          0.097864
reading        0.023535
kinesthetic    0.167625
A              0.013595
B              0.186160
C              0.017653
D              0.103068
E              0.111366
F              0.144832
Name: C_5_2, dtype: float64
expect

c :  C_2_2
c :  C_2_3
c :  C_2_4
X :  A              1.0
B              0.0
C              0.0
D              1.0
E              0.0
F              1.0
audio          1.0
kinesthetic    0.0
reading        0.0
video          1.0
Name: C_2_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              1.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        0.0
video          1.0
Name: C_2_2, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              0.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        0.0
video          1.0
Name: C_2_3, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              0.0
C              0.0
D              0.0
E         

X :  A              0.0
B              1.0
C              1.0
D              1.0
E              0.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          0.0
Name: C_5_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              1.0
E              0.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          0.0
Name: C_5_2, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
arms_payoff :  [1.0, 0.823754471047914]
Index of arm with max payoff :  0
Arm pulled :  C_5_1
Arm Theta:  video          0.037380
audio          0.048746
reading        0.039053
kinesthetic    0.203271
A              0.118325
B              0.164354
C              0.047895
D              0.228258
E              0.051321
F              0.061397
Name: C_5_1, dtype: float64
expected_reward :  0.7621034259740354
Actu

series in record_rounds_data:  A                   1.00000
B                   0.00000
C                   0.00000
D                   1.00000
E                   0.00000
F                   1.00000
audio               0.00000
kinesthetic         1.00000
reading             0.00000
video               0.00000
expected_reward     1.00000
potential_reward    2.80144
reward              1.00000
dtype: float64
c :  C_2_1
c :  C_2_2
c :  C_2_3
c :  C_2_4
X :  A              1.0
B              0.0
C              0.0
D              1.0
E              0.0
F              1.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          0.0
Name: C_2_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              1.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          0.0
Name: C_2_2, dtype: float64
Number of dimensions :  10
mean

X :  A              0.0
B              0.0
C              1.0
D              0.0
E              0.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          0.0
Name: C_4_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              1.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          0.0
Name: C_4_2, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
arms_payoff :  [0.7071067811865476, 0.823754471047914]
Index of arm with max payoff :  1
Arm pulled :  C_4_2
Arm Theta:  video          0.131239
audio          0.096894
reading        0.019927
kinesthetic    0.112527
A              0.087918
B              0.148323
C              0.116487
D              0.111162
E              0.170729
F              0.004795
Name: C_4_2, dtype: float64
expected_reward :  0.635984

...

c :  C_1_2
c :  C_1_3
X :  A              1.0
B              1.0
C              0.0
D              0.0
E              0.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          1.0
Name: C_1_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              1.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          1.0
Name: C_1_2, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              0.0
B              1.0
C              0.0
D              1.0
E              1.0
F              0.0
audio          0.0
kinesthetic    1.0
reading        0.0
video          1.0
Name: C_1_3, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
arms_payoff :  [0.3333026269325189, 0.8378835544657367, 0.29567502400368684]
Index of arm with max pay

series in record_rounds_data:  A                   1.000000
B                   1.000000
C                   0.000000
D                   0.000000
E                   0.000000
F                   0.000000
audio               1.000000
kinesthetic         1.000000
reading             1.000000
video               1.000000
expected_reward     0.200813
potential_reward    0.000000
reward              0.000000
dtype: float64
c :  C_7_1
c :  C_7_2
c :  C_7_3
c :  C_7_4
X :  A              1.0
B              1.0
C              0.0
D              1.0
E              1.0
F              0.0
audio          1.0
kinesthetic    1.0
reading        1.0
video          1.0
Name: C_7_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              0.0
B              0.0
C              0.0
D              0.0
E              0.0
F              0.0
audio          1.0
kinesthetic    1.0
reading        1.0
video          1.0
Name: C_7_2, dtype: float64
Number of dimensio

Actual Reward :  1
series in record_rounds_data:  A                   0.000000
B                   1.000000
C                   0.000000
D                   1.000000
E                   1.000000
F                   0.000000
audio               1.000000
kinesthetic         1.000000
reading             1.000000
video               1.000000
expected_reward     0.371997
potential_reward    0.338065
reward              1.000000
dtype: float64
c :  C_9_1
c :  C_9_2
X :  A              0.0
B              1.0
C              1.0
D              0.0
E              1.0
F              0.0
audio          1.0
kinesthetic    1.0
reading        1.0
video          1.0
Name: C_9_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              0.0
D              0.0
E              0.0
F              0.0
audio          1.0
kinesthetic    1.0
reading        1.0
video          1.0
Name: C_9_2, dtype: float64
Number of dimensions 

X :  A              1.0
B              1.0
C              0.0
D              1.0
E              1.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        1.0
video          1.0
Name: C_7_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              0.0
B              0.0
C              0.0
D              0.0
E              0.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        1.0
video          1.0
Name: C_7_2, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              0.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        1.0
video          1.0
Name: C_7_3, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              1.0
E              1.0
F              0.0
audio

series in record_rounds_data:  A                   1.000000
B                   1.000000
C                   1.000000
D                   0.000000
E                   1.000000
F                   0.000000
audio               1.000000
kinesthetic         0.000000
reading             1.000000
video               1.000000
expected_reward     0.270402
potential_reward    0.000000
reward              1.000000
dtype: float64
c :  C_10_1
c :  C_10_2
X :  A              1.0
B              1.0
C              0.0
D              1.0
E              1.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        1.0
video          1.0
Name: C_10_1, dtype: float64
Number of dimensions :  10
mean : 0.0 type : <class 'numpy.float64'>
X :  A              1.0
B              1.0
C              1.0
D              0.0
E              1.0
F              0.0
audio          1.0
kinesthetic    0.0
reading        1.0
video          1.0
Name: C_10_2, dtype: float64
Number of dimensions :  10
mean : 0.

In [27]:
getRoundsData
# Merging student & content context information

# context = pd.concat([student_context_df ,content_context_df] , axis=1)
# context

In [89]:
# number_of_contexts = len(student_context_df.columns) + len(content_context_df.columns)
# print('number_of_contexts : ', number_of_contexts)
# nature_arm_parameter_df = pd.DataFrame(data = np.random.uniform(size=(len(all_contents) , number_of_contexts)) ,  index = all_contents , dtype= np.float16 )
# #nature_arm_parameter_df

number_of_contexts :  10


In [ ]:
#number_of_features = 10

In [126]:
# class Person:
#     def __init__(self):
#         self.name = 'Aazim'
#         self.age = 25
#         self.status = 'Divorced'

# my_dict = {'person' : Person()}
# p = my_dict['person']
# p.status = 'Married'
# p = my_dict['person']
# p.status

'Married'

In [ ]:
# class Student: 
#     def __init__():
#         self.current_course = ''
#         self.current_topic = ''
    
#     def learn():
        
# class Simulator:
    
#     def __init__():
        
#     def start():
#         for s in number_of_students:
            
        
# sim = Simulator()
# sim.start()

In [40]:
# import numpy as np


# # = np.array([[1,2,3],[4,5,6],[7,8,9]])


# #np.linalg.inv(A)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])